<a href="https://colab.research.google.com/github/snoopuppy582/aiffel_quest_rs/blob/main/resnet_ablationstudy/resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ResNet-34/50 Ablation Study (Plain vs. Residual)
이 노트북은 ResNet-34, ResNet-50의 Plain 모델과 Residual 모델을 구현하고, cats_vs_dogs 데이터셋으로 성능을 비교(Ablation Study)합니다.

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Add, MaxPool2D, GlobalAveragePooling2D, Dense
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

print(f"TensorFlow Version: {tf.__version__}")

TensorFlow Version: 2.19.0


In [9]:
# Ablation Study를 위해 모든 모델에 동일하게 적용
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10  # 루브릭: "동일한 epoch만큼 학습"

In [10]:
# (셀 4: Code 셀 - 3. 데이터셋 준비 (Rubric 2))

# cats_vs_dogs 데이터셋 로드 및 전처리
# [수정됨] data 1개를 받는 대신, image, label 2개를 받도록 수정
def preprocess(image, label):
    # image = data['image'] # (불필요)
    # label = data['label'] # (불필요)

    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0  # 정규화

    return image, label

# tfds에서 데이터 로드 (훈련/검증/테스트 8:1:1로 분할)
(train_ds, val_ds, test_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

# 데이터 파이프라인 구성
train_batches = train_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_batches = val_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("데이터셋 준비 완료")

데이터셋 준비 완료


In [11]:
# (A) ResNet-34용 BasicBlock
def basic_block(x, filters, use_skip=True, stride=1):
    shortcut = x

    # 첫 번째 Conv + BN + ReLU
    f = Conv2D(filters, 3, strides=stride, padding='same')(x)
    f = BatchNormalization()(f)
    f = ReLU()(f)

    # 두 번째 Conv + BN
    f = Conv2D(filters, 3, strides=1, padding='same')(f)
    f = BatchNormalization()(f)

    # --- Skip Connection ---
    if use_skip:
        # 크기가 달라지거나(stride != 1) 채널이 달라지면 1x1 Conv로 shortcut을 맞춰줌
        if stride != 1 or shortcut.shape[-1] != filters:
            shortcut = Conv2D(filters, 1, strides=stride, padding='same')(shortcut)
            shortcut = BatchNormalization()(shortcut)

        f = Add()([f, shortcut]) # f(x) + x
    # -----------------------

    f = ReLU()(f)
    return f

# (B) ResNet-50용 BottleneckBlock
def bottleneck_block(x, filters, use_skip=True, stride=1):
    shortcut = x

    # 1x1 Conv
    f = Conv2D(filters, 1, strides=1, padding='same')(x)
    f = BatchNormalization()(f)
    f = ReLU()(f)

    # 3x3 Conv
    f = Conv2D(filters, 3, strides=stride, padding='same')(f)
    f = BatchNormalization()(f)
    f = ReLU()(f)

    # 1x1 Conv (채널 4배로 확장)
    f = Conv2D(filters * 4, 1, strides=1, padding='same')(f)
    f = BatchNormalization()(f)

    # --- Skip Connection ---
    if use_skip:
        # Bottleneck에서는 항상 채널이 4배로 늘어나므로, shortcut도 맞춰줘야 함
        if stride != 1 or shortcut.shape[-1] != filters * 4:
            shortcut = Conv2D(filters * 4, 1, strides=stride, padding='same')(shortcut)
            shortcut = BatchNormalization()(shortcut)

        f = Add()([f, shortcut]) # f(x) + x
    # -----------------------

    f = ReLU()(f)
    return f

print("ResNet 블록 2종류 구현 완료")

ResNet 블록 2종류 구현 완료


In [12]:
# 블록을 조립하여 ResNet 모델을 만드는 함수
def build_resnet(block_fn, repetitions, use_skip=True):
    inputs = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    # 1. 초기 Conv
    x = Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPool2D(3, strides=2, padding='same')(x)

    # 2. ResNet 블록 스택 (e.g., [3, 4, 6, 3])
    # ... # TODO: ResNet-34와 50의 구조(stage 2~5)를
    # ... # block_fn(Basic/Bottleneck)과 repetitions(블록 개수)를
    # ... # 사용해서 for문으로 쌓아 올리는 코드를 작성하세요.
    # ... # (힌트: stage가 바뀔 때 filters=64, 128, 256, 512 / stride=1, 2, 2, 2)

    filters = 64
    for i, reps in enumerate(repetitions):
        # 첫 번째 블록만 stride=2 적용 (stage 1 제외)
        stride = 2 if i > 0 else 1

        for j in range(reps):
            # 첫 번째 스택의 첫 블록만 stride=1 유지
            current_stride = stride if j == 0 else 1
            x = block_fn(x, filters, use_skip=use_skip, stride=current_stride)

        filters *= 2 # 다음 stage는 filters 2배

    # 3. 분류기 (Classifier)
    x = GlobalAveragePooling2D()(x)

    # cats_vs_dogs는 이진 분류 (1)
    outputs = Dense(1, activation='sigmoid')(x)

    return models.Model(inputs, outputs)

print("ResNet 생성 함수 준비 완료")

ResNet 생성 함수 준비 완료


In [13]:
# ResNet-34, 50의 블록 개수
CONFIG_34 = [3, 4, 6, 3] # BasicBlock
CONFIG_50 = [3, 4, 6, 3] # BottleneckBlock

print("--- 1. Plain-34 (use_skip=False) ---")
plain_34 = build_resnet(basic_block, CONFIG_34, use_skip=False)
# plain_34.summary() # (너무 길어질 수 있으니 필요시 주석 해제)

print("--- 2. Residual-34 (use_skip=True) ---")
resnet_34 = build_resnet(basic_block, CONFIG_34, use_skip=True)
resnet_34.summary() # 루브릭 요구사항이므로 대표 모델 1개 summary 출력

print("--- 3. Plain-50 (use_skip=False) ---")
plain_50 = build_resnet(bottleneck_block, CONFIG_50, use_skip=False)
# plain_50.summary()

print("--- 4. Residual-50 (use_skip=True) ---")
resnet_50 = build_resnet(bottleneck_block, CONFIG_50, use_skip=True)
# resnet_50.summary()

--- 1. Plain-34 (use_skip=False) ---
--- 2. Residual-34 (use_skip=True) ---


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_204 (Conv2D) │ (None, 112, 112,  │      9,472 │ input_layer_5[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_204[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_197 (ReLU)    │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 56, 56,    │          0 │ re_lu_197[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_205 (Conv2D) │ (None, 56, 56,    │     36,928 │ max_pooling2d_5[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_205[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_198 (ReLU)    │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_206 (Conv2D) │ (None, 56, 56,    │     36,928 │ re_lu_198[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_206[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_32 (Add)        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d_5[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_199 (ReLU)    │ (None, 56, 56,    │          0 │ add_32[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_207 (Conv2D) │ (None, 56, 56,    │     36,928 │ re_lu_199[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_207[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_200 (ReLU)    │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_208 (Conv2D) │ (None, 56, 56,    │     36,928 │ re_lu_200[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_208[0][0]

 Total params: 21,310,721 (81.29 MB)

 Trainable params: 21,293,697 (81.23 MB)

 Non-trainable params: 17,024 (66.50 KB)

--- 3. Plain-50 (use_skip=False) ---
--- 4. Residual-50 (use_skip=True) ---


In [14]:
# 4개의 모델을 동일하게 훈련/평가하기 위한 함수
def compile_and_train(model, model_name):
    print(f"\n--- {model_name} 훈련 시작 ---")

    # 1. 컴파일
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy', # 이진 분류
        metrics=['accuracy']
    )

    # 2. 훈련 (동일한 EPOCHS 적용)
    history = model.fit(
        train_batches,
        validation_data=val_batches,
        epochs=EPOCHS,
        verbose=1 # 훈련 과정을 보여줍니다
    )

    # 훈련 중 loss가 감소하는지 확인 (Rubric 2)
    print(f"--- {model_name} 훈련 완료 ---")
    return history

# 훈련 결과를 저장할 딕셔너리
histories = {}

In [ ]:
# [주의] 4개 모델을 훈련하므로 시간이 매우 오래 걸립니다. (EPOCHS를 1~2로 줄여서 테스트 권장)

# 'histories' 딕셔너리에 각 모델의 훈련 기록을 저장합니다.
histories = {}

histories['Plain-34'] = compile_and_train(plain_34, "Plain-34")
histories['Residual-34'] = compile_and_train(resnet_34, "Residual-34")
histories['Plain-50'] = compile_and_train(plain_50, "Plain-50")
histories['Residual-50'] = compile_and_train(resnet_50, "Residual-50")

print("--- [!] 모든 훈련이 완료되었습니다. [!] ---")


--- Plain-34 훈련 시작 ---
Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 173s 239ms/step - accuracy: 0.5137 - loss: 0.7404 - val_accuracy: 0.5331 - val_loss: 0.7061
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 122s 209ms/step - accuracy: 0.5362 - loss: 0.6906 - val_accuracy: 0.5249 - val_loss: 2.2491
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 122s 209ms/step - accuracy: 0.6011 - loss: 0.6579 - val_accuracy: 0.5494 - val_loss: 0.6853
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 122s 209ms/step - accuracy: 0.6356 - loss: 0.6396 - val_accuracy: 0.5404 - val_loss: 0.7076
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 122s 210ms/step - accuracy: 0.5905 - loss: 0.6660 - val_accuracy: 0.5714 - val_loss: 0.6940
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 122s 209ms/step - accuracy: 0.5993 - loss: 0.6582 - val_accuracy: 0.5318 - val_loss: 0.9800
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 122s 210ms/step - accuracy: 0.6182 - loss: 0.6486 - val_accuracy: 0.6019 - val_loss: 0.7652
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 122s 209ms/s

In [ ]:
# 루브릭: "loss가 감소하는 것이 확인되었다."
# 훈련된 모델 중 대표 1개(Residual-34)의 훈련 과정을 시각화합니다.

history = histories['Residual-34']

plt.figure(figsize=(12, 5))

# 1. Loss 그래프
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('ResNet-34 Loss (Residual)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# 2. Accuracy 그래프
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('ResNet-34 Accuracy (Residual)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# (셀 9)의 훈련이 모두 끝난 후 이 셀을 실행하세요.

# 1. 딕셔너리에서 마지막 val_accuracy 값 추출
try:
    p_34_acc = histories['Plain-34'].history['val_accuracy'][-1]
    r_34_acc = histories['Residual-34'].history['val_accuracy'][-1]
    p_50_acc = histories['Plain-50'].history['val_accuracy'][-1]
    r_50_acc = histories['Residual-50'].history['val_accuracy'][-1]
except NameError:
    print("![오류] (셀 9)의 'histories' 딕셔너리가 아직 생성되지 않았습니다.")
    print("![오류] (셀 9)를 먼저 실행하여 4개 모델의 훈련을 완료해주세요.")
except KeyError as e:
    print(f"![오류] histories 딕셔너리에 {e} 키가 없습니다. 훈련이 정상적으로 완료되었는지 확인하세요.")

# 2. Markdown 형식으로 결과표 출력
print("## 🚀 Ablation Study 결과 (Rubric 3)")
print(f"* **Dataset:** `cats_vs_dogs`")
print(f"* **Epochs:** `{EPOCHS}` (모든 모델 동일)")
print(f"* **Metric:** `Validation Accuracy` (마지막 Epoch 기준)")
print("\n" + "="*50 + "\n")
print("| 모델 (Model) | Validation Accuracy |")
print("| :--- | :--- |")
print(f"| Plain ResNet-34 | {p_34_acc:.4f} |")
print(f"| Residual ResNet-34 | {r_34_acc:.4f} |")
print(f"| Plain ResNet-50 | {p_50_acc:.4f} |")
print(f"| Residual ResNet-50 | {r_50_acc:.4f} |")
print("\n" + "="*50)
print("\n[성공] 위 표를 복사하여 GitHub의 README.md 파일에 붙여넣으세요.")